In [1]:
import numpy as np

# -----------------------------
# Parameters (you may change)
# -----------------------------
g = 9.81       # gravity
c = 1e-3           # drag constant
dt = 0.01          # timestep
target_dx = 800.0  # example target distance (meters)
tol = 1e-3         # tolerance for root finding
# -----------------------------

# Drag derivatives
def dvdt(vx, vz):
    v = np.sqrt(vx*vx + vz*vz)
    ax = -c * vx * v
    az = -g - c * vz * v
    return ax, az

# Runge–Kutta trajectory simulation
def simulate(v0, theta):
    vx = v0 * np.cos(theta)
    vz = v0 * np.sin(theta)
    x, z = 0.0, 0.0

    while z >= 0:
        ax, az = dvdt(vx, vz)

        # RK4 integration
        vx1, vz1 = ax, az
        vx2, vz2 = dvdt(vx + 0.5*dt*vx1, vz + 0.5*dt*vz1)
        vx3, vz3 = dvdt(vx + 0.5*dt*vx2, vz + 0.5*dt*vz2)
        vx4, vz4 = dvdt(vx + dt*vx3, vz + dt*vz3)

        vx += dt*(vx1 + 2*vx2 + 2*vx3 + vx4)/6
        vz += dt*(vz1 + 2*vz2 + 2*vz3 + vz4)/6
        x += vx * dt
        z += vz * dt

        # stop if projectile clearly passes the target
        if x > target_dx + 200:
            break

    return x  # horizontal distance when projectile hits ground

# For fixed v0, solve for theta that hits target
def solve_theta(v0):
    low, high = np.deg2rad(1), np.deg2rad(89)

    for _ in range(40):
        mid = 0.5*(low + high)
        x_mid = simulate(v0, mid)

        if abs(x_mid - target_dx) < tol:
            return mid

        # bisection direction
        x_low = simulate(v0, low)
        if (x_low - target_dx)*(x_mid - target_dx) < 0:
            high = mid
        else:
            low = mid

    return mid

# Outer loop: solve for v0 that admits a valid theta
def solve_v0():
    low, high = 10, 3000  # reasonable artillery speeds (m/s)

    for _ in range(40):
        mid = 0.5*(low + high)
        theta = solve_theta(mid)
        x_mid = simulate(mid, theta)

        if abs(x_mid - target_dx) < tol:
            return mid, theta

        if x_mid > target_dx:
            high = mid
        else:
            low = mid

    return mid, theta


# Run solver

v0_sol, theta_sol = solve_v0()

print("\nSolution found:")
print(f"v0    = {v0_sol:.3f} m/s")
print(f"theta = {np.rad2deg(theta_sol):.3f} degrees")


Solution found:
v0    = 1505.000 m/s
theta = 80.391 degrees
